In [1]:
# module
import numpy as np
import pandas as pd
import ast
import nltk
import pickle 

#importing preprocessed file
movies=pickle.load(open('./data/all_movies.pkl','rb'))

In [2]:
# filter our id,userId,rating and title of movies
userId_movieId=movies[['id','userId','rating','title']]

# remove duplicate rows
userId_movieId=userId_movieId.drop_duplicates()

,id,userId,rating,title
0,862,7,3.0,Toy Story
1,862,9,4.0,Toy Story
2,862,13,5.0,Toy Story
3,862,15,2.0,Toy Story
4,862,19,3.0,Toy Story
...,...,...,...,...
100117,390989,624,1.5,Sharknado 4: The 4th Awakens
100118,159550,287,5.0,The Last Brickmaker in America
100119,392572,611,5.0,Rustom
100120,402672,611,3.0,Mohenjo Daro


In [4]:
# create a matrix between id and user id with values of rating and fill na values with 0
movieid_userId_mat=pd.pivot_table(movies,index='id',columns='userId',values='rating').fillna(0)
movieid_userId_mat

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
id,,,,,,,,,,,,,,,,,,,,,
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
9991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# sparse matrix don't contain any 0 values 
from scipy.sparse import csr_matrix
user_movies_sparse=csr_matrix(movieid_userId_mat)

# here alogorithm as brute means first each vector in sparse matrix distance calculate with respect to each vector present in sparse matrix
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

# train model on sparse matrix
model.fit(user_movies_sparse)

# distances are the list of k nearest neighbors and suggestions is list of that k nearest vectors
distances,suggestions=model.kneighbors(movieid_userId_mat.iloc[74,:].values.reshape(1,-1),n_neighbors=20)

In [24]:
# create list of movie id
movie_ids=set(userId_movieId.id)

# list of movies with its recommedation in as pair
movieId_with_recommedations=[]

# iterate over all movie id
for id in movie_ids:
    i=np.where(movieid_userId_mat.index==str(id))[0][0]
    distances,suggestions=model.kneighbors(movieid_userId_mat.iloc[i,:].values.reshape(1,-1),n_neighbors=21)

    # append recommended movies for each movie
    movieId_with_recommedations.append([id,movieid_userId_mat.index[suggestions[0]]])
    
movieId_with_recommedations


[['14919',
  Index(['14919', '4961', '16337', '106837', '10212', '18111', '28774', '11471',
         '8982', '19204', '28165', '18498', '13333', '10553', '17897', '28169',
         '33762', '28605', '29787', '18080', '65460'],
        dtype='object', name='id')],
 ['10587',
  Index(['10587', '11895', '11546', '87428', '256961', '109418', '22317', '9714',
         '10173', '15159', '29621', '9072', '1268', '11825', '9757', '77120',
         '7278', '16299', '18828', '157375', '14361'],
        dtype='object', name='id')],
 ['59726',
  Index(['33908', '11343', '26118', '59726', '25649', '174671', '254191',
         '321258', '39891', '13586', '46591', '18912', '24124', '18939',
         '207774', '47886', '233', '211052', '265010', '39681', '123109'],
        dtype='object', name='id')],
 ['9464',
  Index(['9464', '15555', '10952', '210479', '44629', '14577', '16608', '14785',
         '27845', '10056', '8270', '26038', '8672', '26690', '28219', '1942',
         '21635', '12762', '41397'

In [25]:
import pickle 
# create file in directory
pickle.dump(movieId_with_recommedations,open('movieId_with_recommedations_kNearestNeighbor.pkl','wb'))
pickle.dump(userId_movieId,open('movies_for_kNearestNeighbor.pkl','wb'))